In [1]:
# define matplotlibplotting backend
# %matplotlib -l shows all available backends
%matplotlib qt

In [2]:
import os
import math

import numpy as np

import numpy.polynomial.polynomial as poly

from lib.parameters import Parameters
from lib.plot_builder import PlotBuilder
from lib.h5_reader import H5Reader
from lib.plain_reader import PlainReader

In [3]:
##  configuration options
config_file = '/home/cosmonaut/mount/m37s19/parameters.xml'

cfg = Parameters(config_file)

# shape = [96,384,146,768]
shape = [64,320,96,732]
# shape = [0,320,64,732]
# shape = [0, 0, cfg.number_r_grid, cfg.number_z_grid]
timestamp=2.24e-9
# timestamp=1e-10
show_grid=True
ylim_r=None
ylim_z=None
use_cache=False
verbose=True
specie='electrons'

# separate all velocity range to N equal segments 
# to compare the number of particles in each of them
segments = 500

use_approx=True # use approximation
approx_degree=18 # degree of polynomial approximation

use_directed_filter = True

# 0 - position r                                                                                                                                                                                            
# 1 - position phi                                                                                                                                                                                          
# 2 - position z                                                                                                                                                                                            
# 3 - velocity r                                                                                                                                                                                            
# 4 - velocity phi                                                                                                                                                                                          
# 5 - velocity z
component = 0

autoselect = True

x_axis_label = r'$\mathit{t (s)}$'
y_r_axis_label = r'$\mathit{E_r (\frac{V}{m})}$'
y_z_axis_label = r'$\mathit{E_z (\frac{V}{m})}$'

e_r_plot_name = r'$\mathbf{Electrical\enspace Field\enspace Radial\enspace Component}\enspace(E_r)$'
e_z_plot_name = r'$\mathbf{Electrical\enspace Field\enspace Longitudal\enspace Component}\enspace(E_z)$'
rho_beam_plot_name = r'$\mathbf{Electron\enspace Bunch\enspace Density}\enspace (\rho_{bunch})$'

In [4]:
# define reader (plain reader used)
if not cfg.use_hdf5:
    reader = PlainReader(path = cfg.data_path,
                         data_root=cfg.data_root,
                         fullframe_size=[cfg.number_r_grid , cfg.number_z_grid],
                         fpds=cfg.frames_per_file, 
                         use_cache=use_cache,
                         verbose=verbose)
else:
    reader = H5Reader(str(os.path.join(cfg.data_path, 'data.h5')), use_cache=use_cache)
    reader.verbose = True

In [5]:
data_r = data_phi = data_z = []
pos_r = pos_z = []

for probe in cfg.probes:
    if (probe.type == 'mpframe') and (probe.r_start == shape[0]) and (probe.z_start == shape[1]) and (probe.r_end == shape[2]) and (probe.z_end == shape[3]) and probe.component == specie:
        frame = cfg.get_frame_number_by_timestamp(timestamp, probe.schedule)
        print("Loading frame", frame, probe.schedule, timestamp)
        data_r = reader.get_mpframe(specie, shape, 3, frame)
        data_phi = reader.get_mpframe(specie, shape, 4, frame)
        data_z = reader.get_mpframe(specie, shape, 5, frame)
        
        pos_r = reader.get_mpframe(specie, shape, 0, frame)
        pos_z = reader.get_mpframe(specie, shape, 2, frame)
        
data = np.sqrt(np.power(data_r, 2) + np.power(data_phi, 2) + np.power(data_z, 2))

Loading frame 224 100 2.24e-09


In [6]:
if use_directed_filter:
    # substract directed velocity
    r_grid_amount = int((shape[2]-shape[0]) / 2)
    z_grid_amount = int((shape[3]-shape[1]) / 2)

    subtrahend_r = np.zeros(r_grid_amount * z_grid_amount).reshape(r_grid_amount, z_grid_amount)
    subtrahend_phi = np.zeros(r_grid_amount * z_grid_amount).reshape(r_grid_amount, z_grid_amount)
    subtrahend_z = np.zeros(r_grid_amount * z_grid_amount).reshape(r_grid_amount, z_grid_amount)
    subtrahend = np.zeros(r_grid_amount * z_grid_amount).reshape(r_grid_amount, z_grid_amount)
    count = np.ones(r_grid_amount * z_grid_amount).reshape(r_grid_amount, z_grid_amount)
    
    dr = cfg.r_size / cfg.number_r_grid
    dz = cfg.z_size / cfg.number_z_grid
    r_shift = shape[0] * dr
    z_shift = shape[1] * dz
    
    for i in range(0, r_grid_amount):
        for j in range(0, z_grid_amount):
            for n in range(0, len(pos_r)):
                if pos_r[n] > r_shift + i * dr and pos_r[n] <= r_shift + (i+1) * dr and pos_z[n] > z_shift + j * dz and pos_z[n] <= z_shift + (j+1) * dz:
                    subtrahend_r[i][j] += data_r[n]
                    subtrahend_phi[i][j] += data_phi[n]
                    subtrahend_z[i][j] += data_z[n]
                    count[i][j] += 1
    
    subtrahend_r /= count
    subtrahend_phi /= count
    subtrahend_z /= count
    
    subtrahend = np.sqrt(np.power(subtrahend_r, 2) + np.power(subtrahend_z, 2) + np.power(subtrahend_phi, 2))
       
    # substract directed velocity
    for i in range(0, len(data_r)):
        p_r = pos_r[i]
        p_z = pos_z[i]
        r_cell_number = math.floor((p_r - r_shift) / dr - 1)
        z_cell_number = math.floor((p_z - z_shift) / dz - 1)
        if r_cell_number > r_grid_amount - 1: r_cell_number = r_grid_amount - 1
        if z_cell_number > z_grid_amount - 1: z_cell_number = z_grid_amount - 1
        data_r[i] = data_r[i] - subtrahend_r[r_cell_number][z_cell_number]
        data_phi[i] = data_phi[i] - subtrahend_phi[r_cell_number][z_cell_number]
        data_z[i] = data_z[i] - subtrahend_z[r_cell_number][z_cell_number]
        # data[i] = data[i] - subtrahend[r_cell_number][z_cell_number]
    data = np.sqrt(np.power(data_r, 2) + np.power(data_phi, 2) + np.power(data_z, 2))

In [7]:
velocity_max = np.max(data)
velocity_min = np.min(data)

stat = np.zeros(segments)

slices = np.linspace(velocity_min, velocity_max, segments)

for i in data:
    for j in range(len(slices)-1):
        if i > slices[j] and i <= slices[j+1]:
            stat[j] += 1
            
# stat = stat[stat != 0] # remove zero elements

data_min = np.min(stat)
data_max = np.max(stat)

# normalize to range 0-1
# norm = 1 / data_max
# stat = stat * norm

In [8]:
data_array = np.linspace(data_min, data_max, len(stat))

# define plot builder
plot = PlotBuilder(0, 0, # let the system detects sizes automatically
                   fig_color=cfg.figure_color, 
                   fig_width=cfg.figure_width,
                   fig_height=cfg.figure_height, 
                   fig_dpi=cfg.figure_dpi,
                   font_family=cfg.figure_font_family,
                   font_name=cfg.figure_font_name,
                   font_size=cfg.figure_font_size,
                   tickbox=True, grid=show_grid, is_invert_y_axe=False,
                   aspect='auto', guess_number_ticks=20,
                   # number_x_ticks=10, number_y_ticks=10
                   x_ticklabel_end=1e-9, y_ticklabel_end=1e-9
                  )

# add subplots
plot_r = plot.add_subplot_cartesian_2d(e_r_plot_name, 111, x_axe_label=x_axis_label, y_axe_label=y_r_axis_label)

# set y-limits
if ylim_r is not None:
    plot_r.set_ylim(ylim_r)

if ylim_z is not None:
    plot_z.set_ylim(ylim_z)

In [9]:
# add data
if use_approx:
    x_new = np.linspace(velocity_min, velocity_max, len(stat))

    coefs = poly.polyfit(slices, stat, approx_degree)
    ffit = poly.polyval(x_new, coefs)
    plot_r.plot(x_new, ffit)
    plot_r.plot(slices, stat, linestyle = 'None', marker='.')
else:
    plot_r.plot(slices, stat)

plot.show()

/home/cosmonaut/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RankWarning: The fit may be poorly conditioned
  """
